## Dependências


In [1]:
#Importanando bibliotecas
import os
import cloudpickle
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

## Dados e Constantes

In [2]:
DATA_RAW_PATH = os.path.join('..','data','raw')
DATA_RAW_NAME = 'diabetes_data.csv'
DATA_INTER_PATH = os.path.join('..','data','inter')
DATA_INTER_TEST_NAME = 'test_inter.csv'
TARGET = 'class'

df_raw = pd.read_csv(os.path.join(DATA_RAW_PATH,DATA_RAW_NAME))

## Split dos dados em treino e teste

In [3]:
#Separando os dados. 
X = df_raw.drop(TARGET, axis=1)
y = df_raw[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2,  stratify= y, random_state=123)

In [4]:
#Dados de treino e de teste
df_train = pd.concat([X_train,y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [5]:
df_test.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
342,35,Female,Yes,Yes,No,Yes,No,No,Yes,No,No,No,No,No,No,No,Positive
383,61,Male,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No,Positive
306,28,Male,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,Negative
71,35,Female,No,Yes,Yes,Yes,No,No,No,Yes,No,Yes,Yes,Yes,No,No,Positive
335,53,Male,No,No,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Negative


In [6]:
# salvando os dados de teste
df_test.to_csv(os.path.join(DATA_INTER_PATH,DATA_INTER_TEST_NAME), index=False)
del df_test

## Dataset de treino

In [7]:
#Lendo o dataset de treino.
df_train.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
501,67,Male,No,No,No,No,Yes,No,Yes,No,No,No,No,Yes,No,No,Negative
416,55,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
353,60,Male,No,No,No,Yes,No,No,No,No,No,No,Yes,Yes,No,No,Negative
257,48,Female,Yes,Yes,No,Yes,No,No,Yes,Yes,No,Yes,Yes,No,No,No,Positive
97,31,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,No,Yes,Yes,No,No,Positive


In [8]:
df_train.columns

Index(['Age', 'Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss', 'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring', 'Itching', 'Irritability', 'delayed healing', 'partial paresis', 'muscle stiffness', 'Alopecia', 'Obesity', 'class'], dtype='object')

In [9]:
df_train.shape

(416, 17)

## Limpeza